In [1]:
import os
import pandas as pd
import json
import numpy as np

from typing import List

# Get final results

In [2]:
def parse_solution(filename: str):
    """Parse solution as stored by controller.py"""
    with open(filename, 'r') as f:
        lines = f.readlines()
    if len(lines) == 0:
        return 1000000000, []
    cost = int(lines[1].strip().split(" ")[-1])
    all_routes = []
    for epoch, routes in json.loads(lines[-1].strip()).items():
        all_routes.extend(routes)
    return cost, routes

In [3]:
# Load configurations which map test case indices to settings
with open(os.path.join('instances', 'final', 'configs.json'), 'r') as f:
    configs = json.load(f)

In [4]:
def parse_solution(filename: str):
    with open(filename, 'r') as f:
        lines = f.readlines()
    if len(lines) == 0:
        return 1000000000, []
    cost = int(lines[1].strip().split(" ")[-1])
    all_routes = []
    for epoch, routes in json.loads(lines[-1].strip()).items():
        all_routes.extend(routes)
    return cost, routes

# Create dataframe with the results
results_dir = 'results'
results = []
for team in os.listdir(results_dir):
    for filename in os.listdir(os.path.join(results_dir, team)):
        if filename.endswith(".out"):
            tp, nr = filename.split(".")[0].split("_")[1:]
            nr = int(nr)
            
            cost, routes = parse_solution(
                os.path.join(results_dir, team, filename))

            results.append({
                'team': team,
                'tp': tp,
                'nr': nr,
                'cost': cost,
                'routes': routes,
                **configs[nr]
            })

df = pd.DataFrame(results)
df

,team,tp,nr,cost,routes,instance,instance_seed,static,epoch_tlim
0,OptiML,dynamic,79,315674,"[[532], [534], [536, 538, 535], [537, 533]]",ORTEC-VRPTW-ASYM-a2f023f6-d1-n288-k25.txt,943,False,120
1,OptiML,dynamic,75,269330,"[[579], [565], [572, 566], [574, 560, 563], [5...",ORTEC-VRPTW-ASYM-a5717adc-d1-n207-k12.txt,325,False,120
2,OptiML,dynamic,101,364515,"[[436, 481, 474, 463, 456], [472, 462, 467, 44...",ORTEC-VRPTW-ASYM-a9f1170f-d1-n441-k30.txt,677,False,120
3,OptiML,static,164,146077,"[[17, 141, 107, 59, 87, 20, 108, 86, 122, 18, ...",ORTEC-VRPTW-ASYM-be9d7acd-d1-n221-k15.txt,0,True,300
4,OptiML,static,176,127268,"[[139, 2, 133, 121, 123, 129, 72, 68, 110, 116...",ORTEC-VRPTW-ASYM-bbccd2dd-d1-n271-k25.txt,0,True,300
...,...,...,...,...,...,...,...,...,...
1995,UPB,dynamic,137,355689,"[[571, 572], [568, 566], [575, 578], [564], [5...",ORTEC-VRPTW-ASYM-27232d73-d1-n208-k20.txt,370,False,120
1996,UPB,dynamic,107,442863,"[[437, 438, 435, 442, 432, 422, 434, 419], [43...",ORTEC-VRPTW-ASYM-d3e0d9ae-d1-n212-k15.txt,776,False,120
1997,UPB,static,120,222853,"[[375, 171, 159, 158, 315, 313, 178, 278, 327,...",ORTEC-VRPTW-ASYM-50177145-d1-n378-k30.txt,0,True,600
1998,UPB,static,78,125743,"[[89, 19, 274, 221, 26, 197, 146, 48, 106, 229...",ORTEC-VRPTW-ASYM-a2f023f6-d1-n288-k25.txt,0,True,300


In [5]:
# Compute results per instance with teams in columns
table_per_team = pd.pivot_table(
    df,
    values='cost',
    index=['tp', 'nr'], 
    columns=['team'],
    aggfunc=np.mean
)
table_per_team

team         HowToRoute  HustSmart  Kirchhoffslaw  Kleopatra  MTGBWS  OptiML  \
tp      nr                                                                     
dynamic 1        211178     208168         212022     199921  216306  208758   
        3        351283     337427         342333     326272  341071  336244   
        5        304499     298280         303316     291958  299614  302296   
        7        425328     414694         422912     403610  422708  417905   
        9        368577     355026         369724     348764  360428  360436   
...                 ...        ...            ...        ...     ...     ...   
static  190      152741     152492         152892     152713  152798  152869   
        192      129979     130372         130248     130333  130414  130123   
        194      186132     186165         186226     186165  186165  186165   
        196      131210     131426         131429     131427  131427  131210   
        198      154177     154177         154524     154177  154177  154234   

team         OrbertoHood  Team_sb     UPB      dynamo  
tp      nr                                             
dynamic 1         213219   202946  210341      211445  
        3         340012   338042  337892      333293  
        5         298217   297967  299091  1000000000  
        7         425432   423882  448016      417952  
        9         364533   349459  359111      363426  
...                  ...      ...     ...         ...  
static  190       152407   152492  152769      152601  
        192       130228   130042  130216      130283  
        194       186270   186335  186132      186565  
        196       131163   131427  131210      131210  
        198       154524   154234  154177      154524  

[200 rows x 10 columns]

In [6]:
# Take avg per team and transpose to put teams in rows
table_mean_t = table_per_team.groupby('tp').mean().transpose()

# Compute static and dynamic rank (argsort twice computes rank)
table_mean_t['static_rank'] = np.argsort(np.argsort(table_mean_t['static'])) + 1
table_mean_t['dynamic_rank'] = np.argsort(np.argsort(table_mean_t['dynamic'])) + 1

# Compute avg cost and avg rank
table_mean_t['avg_cost'] = (table_mean_t['dynamic'] + table_mean_t['static']) / 2
table_mean_t['avg_rank'] = (table_mean_t['static_rank'] + table_mean_t['dynamic_rank']) / 2

# Final sorting, first by rank, then by cost as tie-breaker
table_mean_t = table_mean_t.sort_values(['avg_rank', 'avg_cost'])
table_mean_t

tp,dynamic,static,static_rank,dynamic_rank,avg_cost,avg_rank
team,,,,,,
Kleopatra,348831.56,157200.61,2,1,2.530161e+05,1.5
OptiML,359270.09,157188.67,1,3,2.582294e+05,2.0
Team_sb,358161.36,157214.33,3,2,2.576878e+05,2.5
HustSmart,361803.57,157227.24,5,4,2.595154e+05,4.5
MTGBWS,369098.13,157224.13,4,7,2.631611e+05,5.5
OrbertoHood,362481.13,157301.21,9,5,2.598912e+05,7.0
HowToRoute,369797.03,157251.06,7,8,2.635240e+05,7.5
Kirchhoffslaw,370670.53,157249.31,6,9,2.639599e+05,7.5
UPB,367007.49,157322.36,10,6,2.621649e+05,8.0


# Write BKS for static instances

In [7]:
from tools import (
    read_vrplib,
    read_vrptw_solution,
    validate_static_solution
)

Solution = List[List[int]]
def write_solution(path: str, solution: Solution, cost: int):
    """
    Writes a VRP solution to file following the VRPLIB convention.
    """
    with open(path, "w") as fi:
        for idx, route in enumerate(solution, 1):
            fi.write(
                f"Route {idx} : {' '.join([str(s) for s in route])}"
            )
            fi.write("\n")
        fi.write(f"Cost {cost}")
        fi.write("\n")

# Get rows that contain BKS
df_static = df[df['static'] == True]
bks_rows = df_static.loc[df_static.groupby(["instance"])["cost"].idxmin()]

# Loop over BKS and write them in VRPLIB format
for idx, row in bks_rows.iterrows():
    instance_filename = os.path.join('instances', 'final', row['instance'])
    bks_filename = instance_filename.replace(".txt", ".sol")
    
    # Verify solution
    instance = read_vrplib(instance_filename)
    assert row['cost'] == validate_static_solution(instance, row['routes'])
    
    # Write to BKS file
    write_solution(bks_filename, row['routes'], row['cost'])
    
    # Verify BKS file was written correctly
    solution, extra = read_vrptw_solution(bks_filename, return_extra=True)
    assert len(solution) == len(row['routes'])
    assert all(list(r1) == list(r2) for r1, r2 in zip(solution, row['routes']))
    assert int(extra['Cost']) == row['cost']